In [ ]:
import os
import pandas as pd
import pydicom
import h5py
import numpy as np
from scipy import ndimage
import math
import matplotlib.pyplot as plt 

from Functions.data_import import *

### Import patient IDs and labels

In [ ]:
path = "C:/Users/Schule/OneDrive/Studium ZHAW/Bachelorarbeit/Image import/patids_withlabel.xlsx"
patids_labels = pd.read_excel(path)
patids_labels.head()

### Create path file for TOF-Images

In [ ]:
root_dir = "D:\Pat_"
def get_paths_tof(pat_id):
    DIR = root_dir+str(pat_id)+"\Pat"+str(pat_id)+"_TOF"
    folder = os.listdir(DIR)[0]
    DIR = DIR+"\\"+folder
    folder = os.listdir(DIR)[0]
    DIR = DIR+"\\"+folder
    folder = os.listdir(DIR)[0]
    DIR = DIR+"\\"+folder
        
    return(DIR)

In [ ]:
pat_id = 1
DIR = get_paths_tof(pat_id) #get directory for TOF images
files = get_files_in_directory(DIR)
batch_df = pd.DataFrame()
for index, file in enumerate(files):
        DICOM_file = pydicom.read_file(file)

        batch_df.loc[index, 'patient'] = pat_id
        batch_df.loc[index, 'filepath'] = file
        batch_df.loc[index, 'sequence'] = DICOM_file.SequenceName 
        batch_df.loc[index, 'thickness'] = DICOM_file.SliceThickness
        
batch_df

In [ ]:
df = pd.DataFrame()
for i in pat_ids:
    try: 
        batch_df = get_file_list(i)
        df = df.append(batch_df)
    except:
        print(i)

image_info_dir = 'C:/Users/Schule/OneDrive/Studium ZHAW/Bachelorarbeit/Image import/file_paths_tof.csv'
if os.path.exists(image_info_dir):
    os.remove(image_info_dir)
with open(image_info_dir, 'a') as f:
        df.to_csv(f, header = False, index = False)

### Create path file for 3D-Images

In [ ]:
pat_ids_3d2 = (patids_labels["zwei_Bilder"].dropna()).tolist()
pat_ids_3d3 = (patids_labels["drei_Bilder"].dropna()).tolist()
pat_ids_3d0 = (patids_labels["kein_Bild"].dropna()).tolist()
pat_ids_3d1 = [i for i in patids_labels.ID if i not in pat_ids_3d2 and i not in pat_ids_3d3 and i not in pat_ids_3d0]

In [ ]:
def get_paths_3d1(pat_id):
    DIR = root_dir+str(pat_id)+"\Pat"+str(pat_id)+"_3D"
    folder = os.listdir(DIR)[0]
    DIR = DIR+"\\"+folder
    folder = os.listdir(DIR)[0]
    DIR = DIR+"\\"+folder
    folder = os.listdir(DIR)[0]
    DIR = DIR+"\\"+folder
    
    return DIR

In [ ]:
def get_paths_3d2(pat_id):
    DIR1 = root_dir+str(pat_id)+"\Pat"+str(pat_id)+"_3D1"
    folder = os.listdir(DIR1)[0]
    DIR1 = DIR1+"\\"+folder
    folder = os.listdir(DIR1)[0]
    DIR1 = DIR1+"\\"+folder
    folder = os.listdir(DIR1)[0]
    DIR1 = DIR1+"\\"+folder

    ##Folder 3D2
    DIR2 = root_dir+str(pat_id)+"\Pat"+str(pat_id)+"_3D2"
    folder = os.listdir(DIR2)[0]
    DIR2 = DIR2+"\\"+folder
    folder = os.listdir(DIR2)[0]
    DIR2 = DIR2+"\\"+folder
    folder = os.listdir(DIR2)[0]
    DIR2 = DIR2+"\\"+folder
    
    return (DIR1, DIR2)

In [ ]:
def get_paths_3d3(pat_id):
    DIR1 = root_dir+str(pat_id)+"\Pat"+str(pat_id)+"_3D1"
    folder = os.listdir(DIR1)[0]
    DIR1 = DIR1+"\\"+folder
    folder = os.listdir(DIR1)[0]
    DIR1 = DIR1+"\\"+folder
    folder = os.listdir(DIR1)[0]
    DIR1 = DIR1+"\\"+folder

    ##Folder 3D2
    DIR2 = root_dir+str(pat_id)+"\Pat"+str(pat_id)+"_3D2"
    folder = os.listdir(DIR2)[0]
    DIR2 = DIR2+"\\"+folder
    folder = os.listdir(DIR2)[0]
    DIR2 = DIR2+"\\"+folder
    folder = os.listdir(DIR2)[0]
    DIR2 = DIR2+"\\"+folder
    
    ##Folder 3D3
    DIR3 = root_dir+str(pat_id)+"\Pat"+str(pat_id)+"_3D3"
    folder = os.listdir(DIR3)[0]
    DIR3 = DIR3+"\\"+folder
    folder = os.listdir(DIR3)[0]
    DIR3 = DIR3+"\\"+folder
    folder = os.listdir(DIR3)[0]
    DIR3 = DIR3+"\\"+folder
    
    return (DIR1, DIR2, DIR3)

In [ ]:
def get_file_list_3d(pat_id):
    if pat_id in pat_ids_3d1:
        image_dir = get_paths_3d1(pat_id)
        listFilesDICOM = get_files_in_directory(image_dir)
        
    if pat_id in pat_ids_3d2:
        image_dir1, image_dir2 = get_paths_3d2(pat_id)
        listFilesDICOM1 = get_files_in_directory(image_dir1)
        listFilesDICOM2 = get_files_in_directory(image_dir2)
        listFilesDICOM = listFilesDICOM1 + listFilesDICOM2
        
    if pat_id in pat_ids_3d3:
        image_dir1, image_dir2, image_dir3 = get_paths_3d3(pat_id)
        listFilesDICOM1 = get_files_in_directory(image_dir1)
        listFilesDICOM2 = get_files_in_directory(image_dir2)
        listFilesDICOM3 = get_files_in_directory(image_dir3)
        listFilesDICOM = listFilesDICOM1 + listFilesDICOM2 + listFilesDICOM3
        
    batch_df = pd.DataFrame()
    for index, file in enumerate(listFilesDICOM):
        DICOM_file = pydicom.read_file(file)

        try:
            batch_df.loc[index, 'patient'] = pat_id
            batch_df.loc[index, 'instancenumber'] = DICOM_file.InstanceNumber
            batch_df.loc[index, 'seriesInstanceUID'] = DICOM_file.SeriesInstanceUID
            batch_df.loc[index, 'filepath'] = file

        except Exception as e:
            print('file {} cannot be loaded.'.format(file))
            print(e)
            
    return batch_df

In [ ]:
df = pd.DataFrame()
for i in pat_ids:
    try: 
        batch_df = get_file_list_3d(i)
        df = df.append(batch_df)
    except:
        print(i)

image_info_dir = 'D:\\file_paths_3D.csv'
if os.path.exists(image_info_dir):
    os.remove(image_info_dir)
with open(image_info_dir, 'a') as f:
        df.to_csv(f, header = False, index = False)

### Import File with paths to images

In [ ]:
PATH_IMAGE_INFO = 'C:/Users/Schule/OneDrive/Studium ZHAW/Bachelorarbeit/Image import/file_paths_tof.csv'
data_info = pd.read_csv(PATH_IMAGE_INFO)
names = ['pat_id','path', 'sequence', 'thickness']
data_info.columns = names
data_info['pat_id'] = data_info['pat_id'].astype(int)
data_info.head()

### Create HDF5 File

In [ ]:
IMAGE_DIMENSIONS_TOF = (128, 112, 40) #format for images 
first_patient = True

if os.path.exists(PATH_3D_H5):
    os.remove(PATH_3D_H5)
    
patient_list = list(set(data_info.pat_id))
#patient_list = patient_list

with h5py.File(PATH_3D_H5, 'a') as f:
    for patient_number in patient_list:
        patient_data = data_info[data_info.pat_id == patient_number].copy()
        directory = get_paths_tof(patient_number)

        #image to array
        raw_3d_image = image_to_array(patient_number)

        #resize image
        scaled_3d_image = scale_array_3D(raw_3d_image, IMAGE_DIMENSIONS_TOF)

        # Image matrices
        X = scaled_3d_image[np.newaxis, :, :, :]

        # Patient ID's
        pat = format(patient_number, '03d')
        pat = np.string_([pat])

        #path to file with all images for patient
        path = np.string_([directory]) 

        #labels 1 = stroke, 0 = TIA
        Y_pat = patids_labels.label_short.loc[patids_labels.ID == patient_number].values
        Y = np.array(Y_pat)

        ## write to h5py sequentially
        ms = [id for id in IMAGE_DIMENSIONS_TOF]
        ms.insert(0, None)
        ms = tuple(ms)

        if first_patient: ## initialize dataset
            f.create_dataset('X', data = X, maxshape = ms, chunks = True)
            f.create_dataset('stroke', data = Y_pat, maxshape = (None,), chunks = True)
            f.create_dataset('pat', data = pat, maxshape = (None,), chunks = True)
            f.create_dataset('path', data = path, maxshape = (None,), chunks = True)
            first_patient = False

        else: ## append dataset
            f['X'].resize((f['X'].shape[0] + X.shape[0]), axis = 0)
            f['X'][-X.shape[0]:, :, :, :] = X
            f['stroke'].resize((f['stroke'].shape[0] + Y_pat.shape[0]), axis = 0)
            f['stroke'][-Y_pat.shape[0]:] = Y_pat
            f['pat'].resize((f['pat'].shape[0] + pat.shape[0]), axis = 0)
            f['pat'][-pat.shape[0]:] = pat
            f['path'].resize((f['path'].shape[0] + path.shape[0]), axis = 0)
            f['path'][-path.shape[0]:] = path

In [ ]:
f.close()

### Checking HDF5 File

In [ ]:
PATH_3D_H5 = 'D:\\tof_data_128_112_40.hdf5'

In [ ]:
dd = h5py.File(PATH_3D_H5, 'r')
list(dd.keys())

In [ ]:
X = dd['X']
np.max(X)

In [ ]:
patients = [p.decode() for p in dd['pat']]
len(patients)

In [ ]:
path = [p for p in dd['path']]
path[0:10]

In [ ]:
labels = [p for p in dd['stroke']]
len(labels)

In [ ]:
dd.close()